In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_hf.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)
        
        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(classifier.train(
                X_train=train_x,
                y_train=train_y,
                X_eval=test_x,
                y_eval=test_y,
                categories=categories,
                epochs=50
            ))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(classifier.train(
                X_train=tx[train_i],
                y_train=ty[train_i],
                X_eval=te_x[test_i],
                y_eval=te_y[test_i],
                categories=categories,
                epochs=50
            ))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class ShinContextualModel:
    '''
    Implementation proposal of: http://milab.snu.ac.kr/pub/BigComp2018.pdf
    '''
    def __init__(self, embedding, conv_size=3, num_layers=2):
        '''Constructor.
        # Parameters:
        embedding: numpy array representing the embedding.
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        num_layers: Number of recurrent convolutions.
        '''
        self._embedding = embedding
        self._conv_size = conv_size
        self._num_layers = num_layers

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size, self._num_layers, self._embedding_tf)
        self._pooling_tf = self._create_maxpooling_layer(self._convolution_tf)

        return self._pooling_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('pool:', str(self._pooling_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self, conv_size, num_layers, embedding):
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, filter_width]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[filter_width]))

        z = embedding
        for _ in range(num_layers):
            conv = tf.nn.conv1d(
                value=z,
                filters=W,
                stride=1,
                padding='SAME')
            bias = tf.nn.bias_add(conv, b)
            c = tf.nn.relu(bias)

            d = tf.nn.dropout(c, 0.75)
            # Add BatchNormalization or LocalResponseNormalization
            e = tf.expand_dims(d, 1)

            z = tf.nn.local_response_normalization(
                e,
                depth_radius=5,
                bias=1,
                alpha=0.001,
                beta=0.75
            )
            z = tf.squeeze(z, 1)
        # endfor
        return z

    def _create_maxpooling_layer(self, convolution):
        conv_size = convolution.shape[1].value
        embedding_size = convolution.shape[2].value

        convolution = tf.expand_dims(convolution, -1)
        pooled = tf.nn.max_pool(
            value=convolution,
            ksize=[1, conv_size, 1, 1],
            strides=[1, 1, 1, 1],
            padding='VALID')

        flat = tf.reshape(pooled, [-1, embedding_size])
        return flat

In [8]:
word_vector = embedding.astype('float32')
model = ShinContextualModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 41)
embedding: (?, 41, 100)
conv: (?, 41, 100)
pool: (?, 100)
output: (?, 10)


In [9]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data, test_data = dataset_split(info)
    test_acc_split.append(train_split_data(train_data, test_data, split_type))

random
Fold:  1
Epoch: 1, Train Loss:  1.789, Train Acc: 38.19%, Val Loss:  1.103, Val Acc: 63.89%, Time: 36.32s *
Epoch: 2, Train Loss: 0.9199, Train Acc: 70.38%, Val Loss: 0.7441, Val Acc: 76.50%, Time: 36.23s *
Epoch: 3, Train Loss: 0.6066, Train Acc: 81.15%, Val Loss: 0.5733, Val Acc: 82.55%, Time: 36.23s *
Epoch: 4, Train Loss: 0.4345, Train Acc: 86.60%, Val Loss: 0.4919, Val Acc: 85.26%, Time: 36.40s *
Epoch: 5, Train Loss:  0.327, Train Acc: 89.81%, Val Loss: 0.4568, Val Acc: 85.61%, Time: 36.37s *
Epoch: 6, Train Loss: 0.2551, Train Acc: 92.24%, Val Loss: 0.4196, Val Acc: 88.11%, Time: 36.37s *
Epoch: 7, Train Loss: 0.2092, Train Acc: 93.49%, Val Loss: 0.4241, Val Acc: 88.18%, Time: 36.31s *
Epoch: 8, Train Loss: 0.1664, Train Acc: 94.88%, Val Loss: 0.4077, Val Acc: 88.18%, Time: 36.36s *
Epoch: 9, Train Loss: 0.1276, Train Acc: 95.99%, Val Loss: 0.4085, Val Acc: 87.61%, Time: 36.45s 
Epoch: 10, Train Loss: 0.1075, Train Acc: 96.70%, Val Loss:  0.392, Val Acc: 89.53%, Time: 36.

Epoch: 1, Train Loss:  1.907, Train Acc: 33.01%, Val Loss:  1.308, Val Acc: 55.63%, Time: 36.37s *
Epoch: 2, Train Loss: 0.9597, Train Acc: 69.73%, Val Loss: 0.7987, Val Acc: 75.57%, Time: 36.38s *
Epoch: 3, Train Loss: 0.6216, Train Acc: 80.46%, Val Loss: 0.6309, Val Acc: 80.27%, Time: 36.40s *
Epoch: 4, Train Loss: 0.4472, Train Acc: 86.46%, Val Loss: 0.5782, Val Acc: 82.98%, Time: 36.38s *
Epoch: 5, Train Loss: 0.3412, Train Acc: 89.44%, Val Loss: 0.4817, Val Acc: 87.11%, Time: 36.41s *
Epoch: 6, Train Loss: 0.2653, Train Acc: 92.00%, Val Loss:  0.459, Val Acc: 86.89%, Time: 36.34s 
Epoch: 7, Train Loss: 0.2034, Train Acc: 93.85%, Val Loss: 0.4458, Val Acc: 86.97%, Time: 36.28s 
Epoch: 8, Train Loss: 0.1638, Train Acc: 95.10%, Val Loss: 0.4045, Val Acc: 88.75%, Time: 36.34s *
Epoch: 9, Train Loss: 0.1348, Train Acc: 95.80%, Val Loss: 0.4237, Val Acc: 88.32%, Time: 36.29s 
Epoch: 10, Train Loss: 0.1091, Train Acc: 96.68%, Val Loss: 0.4215, Val Acc: 87.89%, Time: 36.31s 
Epoch: 11, Tr

Epoch: 1, Train Loss:  1.979, Train Acc: 32.44%, Val Loss:  1.333, Val Acc: 56.74%, Time: 36.40s *
Epoch: 2, Train Loss: 0.9899, Train Acc: 68.30%, Val Loss: 0.8489, Val Acc: 74.63%, Time: 36.40s *
Epoch: 3, Train Loss: 0.6445, Train Acc: 80.22%, Val Loss: 0.6939, Val Acc: 78.69%, Time: 36.48s *
Epoch: 4, Train Loss: 0.4629, Train Acc: 86.00%, Val Loss: 0.5718, Val Acc: 82.39%, Time: 36.48s *
Epoch: 5, Train Loss: 0.3424, Train Acc: 89.42%, Val Loss: 0.5289, Val Acc: 83.54%, Time: 36.47s *
Epoch: 6, Train Loss:   0.26, Train Acc: 92.10%, Val Loss: 0.5115, Val Acc: 85.60%, Time: 36.44s *
Epoch: 7, Train Loss: 0.2042, Train Acc: 93.64%, Val Loss:  0.475, Val Acc: 86.89%, Time: 36.42s *
Epoch: 8, Train Loss: 0.1616, Train Acc: 95.21%, Val Loss: 0.5037, Val Acc: 86.67%, Time: 36.49s 
Epoch: 9, Train Loss:  0.133, Train Acc: 95.91%, Val Loss: 0.5016, Val Acc: 87.24%, Time: 36.51s *
Epoch: 10, Train Loss: 0.1131, Train Acc: 96.42%, Val Loss: 0.5822, Val Acc: 86.10%, Time: 36.50s 
Epoch: 11, 

Epoch: 1, Train Loss:  1.874, Train Acc: 35.72%, Val Loss:   1.27, Val Acc: 57.95%, Time: 36.47s *
Epoch: 2, Train Loss: 0.9428, Train Acc: 70.41%, Val Loss: 0.7886, Val Acc: 75.27%, Time: 36.27s *
Epoch: 3, Train Loss: 0.6295, Train Acc: 80.86%, Val Loss:  0.651, Val Acc: 79.76%, Time: 36.35s *
Epoch: 4, Train Loss: 0.4616, Train Acc: 85.97%, Val Loss: 0.5352, Val Acc: 84.39%, Time: 36.37s *
Epoch: 5, Train Loss: 0.3478, Train Acc: 89.51%, Val Loss: 0.4294, Val Acc: 86.60%, Time: 36.27s *
Epoch: 6, Train Loss: 0.2665, Train Acc: 91.81%, Val Loss: 0.4199, Val Acc: 87.38%, Time: 36.32s *
Epoch: 7, Train Loss: 0.2123, Train Acc: 93.50%, Val Loss: 0.4234, Val Acc: 88.03%, Time: 36.31s *
Epoch: 8, Train Loss: 0.1775, Train Acc: 94.79%, Val Loss: 0.4389, Val Acc: 87.10%, Time: 36.39s 
Epoch: 9, Train Loss: 0.1391, Train Acc: 95.67%, Val Loss: 0.4125, Val Acc: 88.67%, Time: 36.43s *
Epoch: 10, Train Loss:  0.114, Train Acc: 96.50%, Val Loss: 0.4204, Val Acc: 89.45%, Time: 36.41s *
Epoch: 11,

Epoch: 1, Train Loss:   1.86, Train Acc: 35.27%, Val Loss:  1.279, Val Acc: 58.20%, Time: 36.83s *
Epoch: 2, Train Loss: 0.9778, Train Acc: 68.74%, Val Loss:  1.161, Val Acc: 65.70%, Time: 36.77s *
Epoch: 3, Train Loss: 0.6749, Train Acc: 79.25%, Val Loss:  1.091, Val Acc: 69.14%, Time: 36.74s *
Epoch: 4, Train Loss: 0.4957, Train Acc: 84.81%, Val Loss:  1.133, Val Acc: 70.23%, Time: 36.68s *
Epoch: 5, Train Loss: 0.3646, Train Acc: 89.03%, Val Loss:  1.165, Val Acc: 71.72%, Time: 36.64s *
Epoch: 6, Train Loss: 0.2845, Train Acc: 91.36%, Val Loss:  1.298, Val Acc: 71.02%, Time: 36.73s 
Epoch: 7, Train Loss: 0.2272, Train Acc: 92.97%, Val Loss:  1.334, Val Acc: 72.89%, Time: 36.79s *
Epoch: 8, Train Loss: 0.1872, Train Acc: 94.32%, Val Loss:  1.445, Val Acc: 70.78%, Time: 36.81s 
Epoch: 9, Train Loss: 0.1443, Train Acc: 95.36%, Val Loss:   1.57, Val Acc: 70.70%, Time: 36.68s 
Epoch: 10, Train Loss: 0.1186, Train Acc: 96.27%, Val Loss:  1.768, Val Acc: 70.31%, Time: 36.66s 
Epoch: 11, Tr

...

Epoch: 4, Train Loss: 0.8239, Train Acc: 74.10%, Val Loss:  1.187, Val Acc: 63.81%, Time: 36.47s *
Epoch: 5, Train Loss: 0.6553, Train Acc: 79.56%, Val Loss:  1.197, Val Acc: 64.60%, Time: 36.40s *
Epoch: 6, Train Loss: 0.5364, Train Acc: 83.36%, Val Loss:   1.23, Val Acc: 64.95%, Time: 36.40s *
Epoch: 7, Train Loss: 0.4322, Train Acc: 86.58%, Val Loss:  1.325, Val Acc: 66.02%, Time: 36.39s *
Epoch: 8, Train Loss: 0.3537, Train Acc: 88.84%, Val Loss:  1.377, Val Acc: 65.52%, Time: 36.43s 
Epoch: 9, Train Loss: 0.2899, Train Acc: 91.08%, Val Loss:  1.453, Val Acc: 65.74%, Time: 36.49s 
Epoch: 10, Train Loss: 0.2349, Train Acc: 92.88%, Val Loss:  1.625, Val Acc: 66.17%, Time: 36.41s *
Epoch: 11, Train Loss: 0.1985, Train Acc: 93.90%, Val Loss:  1.694, Val Acc: 66.17%, Time: 36.46s *
Epoch: 12, Train Loss: 0.1638, Train Acc: 95.14%, Val Loss:  1.817, Val Acc: 66.88%, Time: 36.46s *
Epoch: 13, Train Loss: 0.1434, Train Acc: 95.66%, Val Loss:  1.957, Val Acc: 65.95%, Time: 36.57s 
Epoch: 14

Epoch: 5, Train Loss: 0.6313, Train Acc: 80.26%, Val Loss:  1.198, Val Acc: 65.04%, Time: 36.73s *
Epoch: 6, Train Loss:  0.515, Train Acc: 84.11%, Val Loss:  1.229, Val Acc: 65.40%, Time: 36.64s *
Epoch: 7, Train Loss: 0.4184, Train Acc: 87.32%, Val Loss:  1.218, Val Acc: 66.17%, Time: 36.70s *
Epoch: 8, Train Loss: 0.3495, Train Acc: 89.45%, Val Loss:   1.29, Val Acc: 65.96%, Time: 36.79s 
Epoch: 9, Train Loss: 0.2828, Train Acc: 91.53%, Val Loss:  1.306, Val Acc: 66.38%, Time: 36.78s *
Epoch: 10, Train Loss: 0.2303, Train Acc: 93.08%, Val Loss:  1.422, Val Acc: 65.47%, Time: 36.83s 
Epoch: 11, Train Loss: 0.1804, Train Acc: 94.83%, Val Loss:  1.517, Val Acc: 65.47%, Time: 36.75s 
Epoch: 12, Train Loss: 0.1552, Train Acc: 95.13%, Val Loss:  1.498, Val Acc: 66.53%, Time: 36.72s *
Epoch: 13, Train Loss: 0.1366, Train Acc: 95.80%, Val Loss:   1.61, Val Acc: 66.67%, Time: 36.74s *
Epoch: 14, Train Loss: 0.1127, Train Acc: 96.64%, Val Loss:  1.762, Val Acc: 65.32%, Time: 36.67s 
Epoch: 15

Epoch: 1, Train Loss:  1.935, Train Acc: 33.51%, Val Loss:  1.694, Val Acc: 44.82%, Time: 34.79s *
Epoch: 2, Train Loss:  1.028, Train Acc: 66.40%, Val Loss:  1.565, Val Acc: 56.88%, Time: 34.64s *
Epoch: 3, Train Loss: 0.6933, Train Acc: 77.96%, Val Loss:  1.592, Val Acc: 61.13%, Time: 34.71s *
Epoch: 4, Train Loss: 0.5143, Train Acc: 84.10%, Val Loss:  1.635, Val Acc: 61.82%, Time: 34.63s *
Epoch: 5, Train Loss: 0.3889, Train Acc: 88.09%, Val Loss:  1.778, Val Acc: 62.84%, Time: 34.68s *
Epoch: 6, Train Loss: 0.2949, Train Acc: 91.06%, Val Loss:  1.889, Val Acc: 62.40%, Time: 34.61s 
Epoch: 7, Train Loss: 0.2337, Train Acc: 92.91%, Val Loss:  1.896, Val Acc: 64.26%, Time: 34.72s *
Epoch: 8, Train Loss: 0.1871, Train Acc: 94.47%, Val Loss:  2.052, Val Acc: 63.96%, Time: 34.71s 
Epoch: 9, Train Loss: 0.1509, Train Acc: 95.64%, Val Loss:  2.127, Val Acc: 64.06%, Time: 34.73s 
Epoch: 10, Train Loss: 0.1188, Train Acc: 96.48%, Val Loss:  2.247, Val Acc: 65.23%, Time: 34.72s *
Epoch: 11, T

Epoch: 1, Train Loss:   2.02, Train Acc: 30.26%, Val Loss:  1.519, Val Acc: 50.38%, Time: 36.69s *
Epoch: 2, Train Loss:  1.023, Train Acc: 66.92%, Val Loss:  1.277, Val Acc: 62.05%, Time: 36.50s *
Epoch: 3, Train Loss: 0.6774, Train Acc: 79.07%, Val Loss:  1.284, Val Acc: 64.86%, Time: 36.58s *
Epoch: 4, Train Loss: 0.5054, Train Acc: 84.21%, Val Loss:  1.322, Val Acc: 63.69%, Time: 36.57s 
Epoch: 5, Train Loss: 0.3764, Train Acc: 88.59%, Val Loss:  1.442, Val Acc: 63.69%, Time: 36.54s 
Epoch: 6, Train Loss: 0.2899, Train Acc: 91.11%, Val Loss:  1.442, Val Acc: 64.93%, Time: 36.50s *
Epoch: 7, Train Loss:  0.228, Train Acc: 92.96%, Val Loss:  1.552, Val Acc: 65.68%, Time: 36.54s *
Epoch: 8, Train Loss: 0.1766, Train Acc: 94.71%, Val Loss:  1.699, Val Acc: 63.21%, Time: 36.51s 
Epoch: 9, Train Loss: 0.1489, Train Acc: 95.61%, Val Loss:  2.021, Val Acc: 62.73%, Time: 36.43s 
Epoch: 10, Train Loss: 0.1224, Train Acc: 96.35%, Val Loss:  2.108, Val Acc: 64.17%, Time: 36.54s 
Epoch: 11, Tra

Epoch: 3, Train Loss: 0.6815, Train Acc: 78.85%, Val Loss:  1.201, Val Acc: 72.31%, Time: 38.76s *
Epoch: 4, Train Loss: 0.5006, Train Acc: 84.56%, Val Loss:  1.085, Val Acc: 74.41%, Time: 38.70s *
Epoch: 5, Train Loss: 0.3726, Train Acc: 88.76%, Val Loss:  1.153, Val Acc: 75.38%, Time: 38.63s *
Epoch: 6, Train Loss: 0.2959, Train Acc: 91.00%, Val Loss:  1.173, Val Acc: 74.83%, Time: 38.51s 
Epoch: 7, Train Loss: 0.2314, Train Acc: 93.24%, Val Loss:  1.299, Val Acc: 72.45%, Time: 38.60s 
Epoch: 8, Train Loss: 0.1967, Train Acc: 94.12%, Val Loss:   1.33, Val Acc: 73.85%, Time: 38.62s 
Epoch: 9, Train Loss: 0.1502, Train Acc: 95.44%, Val Loss:  1.485, Val Acc: 72.17%, Time: 38.66s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.52      0.92      0.66        61
                   Filter       0.84      0.65      0.73        82
    Compute Derived Value